In [ ]:
!python --version

In [ ]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
import numpy as np
import glob
import pathlib
import PIL
import os
import tarfile
import cv2
import numpy as np

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [ ]:
tar_files = ['part1.tar.gz', 'part2.tar.gz', 'part3.tar.gz']
output_folder = os.getcwd() + '/all_images'

if not os.path.exists(output_folder):
    os.makedirs(output_folder)

    for tar_file in tar_files:
        with tarfile.open(tar_file, 'r:gz') as tar:
            for member in tar.getmembers():
                # Remove the leading directory name from the member's name
                member.name = os.path.basename(member.name)
                tar.extract(member, output_folder)

In [ ]:
data_dir = pathlib.Path('.//all_images')

In [ ]:
image_count = len(list(data_dir.glob('**/*.jpg')))
print(image_count)

In [ ]:
age_1 = list(data_dir.glob('1_*'))
PIL.Image.open(str(age_1[1]))

In [ ]:
age_images_dict = {}

for i in range(1, 6):
    age_images_dict[i] = list(data_dir.glob(str(i)+'_*'))

# some ages are missing 

In [ ]:
ages_labels_dict = {}
for i in range(1, 6):
    ages_labels_dict[i] = i-1

In [ ]:
# img = cv2.imread(str(age_images_dict[3][0]))
# img.shape

In [ ]:
x, y = [], []

for age_number, images in age_images_dict.items():
    for image in images:
        img = cv2.imread(str(image))
        resized_img = cv2.resize(img,(180,180))
        x.append(resized_img)
        y.append(ages_labels_dict[age_number])
        
x = np.array(x)/255

In [ ]:
np.savez( os.getcwd() + '/ age_image_data.npz', x, y)

# # run from here if saved .npz file

In [ ]:
with np.load(os.getcwd() + '/ age_image_data.npz') as data:
    x = data['arr_0']
    y = data['arr_1']

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, random_state=0)

In [ ]:
num_classes = len(np.unique(y))

model = Sequential([
  layers.Conv2D(16, 3, padding='same', activation='relu', input_shape = (180,180,3)),
  layers.MaxPooling2D(),
    
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
    
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
    
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
              
model.fit(X_train, y_train, epochs=30)  

In [ ]:
model.evaluate(X_test,y_test)